<span STYLE="font-size:150%"> 
    Segment cylinder regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

In [ ]:
import pyslicer as ps
import slicer
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
sample_name = 'GB3_8W'
segmented_dir = 'segmented_volumes/'
volume_file = 'microCT_volume/microCT_volume.nrrd'
bone_cylinders_dir = 'segmented_volumes/bone_cylinders/'
file_segmentation = 'segmented_volumes/Bone.seg.nrrd'

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Get notebook directory name

In [ ]:
sample_name

# Create segmentationNode

## Create segmentation-related nodes

Create segmentation node

In [ ]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Load `Bone_Cylinder-Middle` segmentation to segmentation node

Load segmentation from .nrrd file as labelmap node slicer util module to load from File [github](https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/util.py#L341-L344)

In [ ]:
file_labelmap =  'Bone_Cylinder-Middle.seg.nrrd'

In [ ]:
name_labelmap = file_labelmap.split('.')[0]
name_labelmap

In [ ]:
# Load segmentation from .seg.nrrd file (includes segment names and colors)

labelmap = slicer.util.loadNodeFromFile(bone_cylinders_dir + file_labelmap, 'VolumeFile', properties={'labelmap':True})
# segmentationNode = slicer.util.loadSegmentation(file_nuclei_segmentation)

Import bone labelmapNode to segmentationNode

In [ ]:
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap, segmentationNode)

Rename last segment of the segmentation node 

In [ ]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(segmentationNode.GetSegmentation().GetNumberOfSegments() - 1)
segment.SetName(name_labelmap)

Delete bone labelmapNode

In [ ]:
slicer.mrmlScene.RemoveNode(labelmap)
# masterVolumeNode.SetDisplayVisibility(1)

In [ ]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

# Compute bone ingrowth volumes

Compute segment statistics

In [ ]:
stats = ps.segmentation.segment_statistics(segmentationNode)

Store volume of each segment

In [ ]:
segment_names = []
volumes_ingrowth_mm3 = []

# Display volume of each segment
for segmentId in stats["SegmentIDs"]:
    
    volume_mm3 = stats[segmentId,"LabelmapSegmentStatisticsPlugin.volume_mm3"]
    volumes_ingrowth_mm3.append(volume_mm3)
    
    segmentName = segmentationNode.GetSegmentation().GetSegment(segmentId).GetName()
    segment_names.append(segmentName)

data_dict = {'segmentName': segment_names, 'volume_mm3':volumes_ingrowth_mm3}
df_stats = pd.DataFrame(data_dict)
df_stats.set_index('segmentName',inplace=True)
df_stats

Define output table

In [ ]:
data = {'ingrowth_middle' : [df_stats.loc[['Bone_Cylinder-Middle']]['volume_mm3'].iloc[0]]
       }

In [ ]:
df_volumes = pd.read_csv(segmented_dir + sample_name + '_mineralized_volumes.csv')
df_2 = pd.DataFrame.from_dict(data)

df = df_volumes.join(df_2)

In [ ]:
outputfile = segmented_dir + sample_name + '_mineralized_volumes.csv'
df.to_csv(outputfile, index=False)